# Full supervised training

To establish a goal line the Inception network is trained in a full supervised fashion.


In [2]:
from deepal_for_ecg.data.load import PTBXLDataLoader
from deepal_for_ecg.data.active_learning import PTBXLActiveLearningDataModule
from deepal_for_ecg.models.inception_network import InceptionNetworkBuilder, InceptionNetworkConfig
from deepal_for_ecg.trainer import PTBXLTrainer
from deepal_for_ecg.util import improve_gpu_capacity

In [3]:
improve_gpu_capacity()

2024-02-19 01:47:45.173993: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 01:47:45.499529: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 01:47:45.499646: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


## Data

Load the data and prepare the data module.

In [4]:
data_loader = PTBXLDataLoader(load_saved_data=True, saved_data_base_dir="../data/saved/")
data_loader.load_data()

In [5]:
data_module = PTBXLActiveLearningDataModule(
    train_samples=data_loader.X_train, 
    test_samples=data_loader.X_test, 
    val_samples=data_loader.X_valid, 
    train_labels_12sl=data_loader.Y_train_12sl, 
    train_labels_ptb_xl=data_loader.Y_train_ptb_xl,
    test_labels=data_loader.Y_test,
    val_labels=data_loader.Y_valid
)

In [9]:
data_module_state = data_module.state_dict()
data_module.update_annotations(buy_idx_ptb_xl=data_module_state["unlabeled_indices"], buy_idx_12sl=set())

## Inception network

### Build the models

Build the Inception networks.

In [11]:
builder = InceptionNetworkBuilder()
config = InceptionNetworkConfig()

ensemble = [builder.build_model(config) for _ in range(5)]

2024-02-19 02:06:41.936488: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 02:06:41.936964: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 02:06:41.937065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 02:06:42.514253: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 02:06:42.514302: I tensorflow/core/common_runtime/gpu/gpu

### Training
Train all five models of the Inception network.

In [19]:
num_classes = 93
trainer = PTBXLTrainer(ensemble[0], model_name="full_supervised_inception_1", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

2024-02-19 02:11:25.352157: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1672128000 exceeds 10% of free system memory.
2024-02-19 02:11:42.558963: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1672128000 exceeds 10% of free system memory.
2024-02-19 02:11:43.109038: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 52392000 exceeds 10% of free system memory.
2024-02-19 02:11:43.171255: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 52392000 exceeds 10% of free system memory.
2024-02-19 02:11:43.221907: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 52392000 exceeds 10% of free system memory.
2024-02-19 02:11:44.193614: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
Epoch: 0:   0%|          | 0/69 [00:00<?, ?it/s]2024-02-19 02:11:51.672961: I external/local_xla/xla/stream_exec

Saving new best model with AUC: 0.7692109942436218
Loss: 0.2529252767562866, Accuracy: 91.01961517333984, AUC: 63.07650375366211, Validation Loss: 0.19528336822986603, Validation Accuracy: 93.57991027832031, Validation AUC: 76.92109680175781


Epoch: 1: 100%|██████████| 69/69 [00:10<00:00,  6.68it/s]


Saving new best model with AUC: 0.8035593628883362
Loss: 0.1386144459247589, Accuracy: 95.17579650878906, AUC: 78.6104965209961, Validation Loss: 0.15856751799583435, Validation Accuracy: 94.42219543457031, Validation AUC: 80.3559341430664


Epoch: 2: 100%|██████████| 69/69 [00:10<00:00,  6.70it/s]


...

Loss: 0.06804069876670837, Accuracy: 97.51319885253906, AUC: 93.43726348876953, Validation Loss: 0.13508640229701996, Validation Accuracy: 95.16252136230469, Validation AUC: 87.94464111328125


Epoch: 48: 100%|██████████| 69/69 [00:11<00:00,  6.25it/s]


Loss: 0.06909879297018051, Accuracy: 97.4716567993164, AUC: 93.27274322509766, Validation Loss: 0.10924036055803299, Validation Accuracy: 96.19986724853516, Validation AUC: 87.81880950927734


Epoch: 49: 100%|██████████| 69/69 [00:11<00:00,  6.26it/s]


Loss: 0.0654389038681984, Accuracy: 97.61518096923828, AUC: 93.45406341552734, Validation Loss: 0.11264216899871826, Validation Accuracy: 96.14321899414062, Validation AUC: 87.16510772705078


In [20]:
trainer = PTBXLTrainer(ensemble[1], model_name="full_supervised_inception_2", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

Epoch: 0: 100%|██████████| 69/69 [00:21<00:00,  3.20it/s]


Saving new best model with AUC: 0.7365515828132629
Loss: 0.25416165590286255, Accuracy: 90.99275207519531, AUC: 64.2433090209961, Validation Loss: 0.22151218354701996, Validation Accuracy: 92.40464782714844, Validation AUC: 73.65515899658203


Epoch: 1: 100%|██████████| 69/69 [00:10<00:00,  6.63it/s]


Saving new best model with AUC: 0.7800142168998718
Loss: 0.13595597445964813, Accuracy: 95.18246459960938, AUC: 79.03479766845703, Validation Loss: 0.1647363305091858, Validation Accuracy: 93.9384994506836, Validation AUC: 78.00141906738281


Epoch: 2: 100%|██████████| 69/69 [00:10<00:00,  6.69it/s]


...

Loss: 0.07025323808193207, Accuracy: 97.41560363769531, AUC: 92.71440124511719, Validation Loss: 0.11023534834384918, Validation Accuracy: 96.06588745117188, Validation AUC: 88.36750030517578


Epoch: 48: 100%|██████████| 69/69 [00:10<00:00,  6.66it/s]


Loss: 0.0683346688747406, Accuracy: 97.50338745117188, AUC: 92.8779525756836, Validation Loss: 0.11058307439088821, Validation Accuracy: 95.9993896484375, Validation AUC: 87.96160125732422


Epoch: 49: 100%|██████████| 69/69 [00:10<00:00,  6.65it/s]


Loss: 0.06766028702259064, Accuracy: 97.55079650878906, AUC: 93.12366485595703, Validation Loss: 0.11276347190141678, Validation Accuracy: 95.9314193725586, Validation AUC: 88.25331115722656


In [21]:
trainer = PTBXLTrainer(ensemble[2], model_name="full_supervised_inception_3", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

Epoch: 0: 100%|██████████| 69/69 [00:19<00:00,  3.61it/s]


Saving new best model with AUC: 0.7597726583480835
Loss: 0.2582937777042389, Accuracy: 91.0049819946289, AUC: 63.35000991821289, Validation Loss: 0.2113575041294098, Validation Accuracy: 93.1474380493164, Validation AUC: 75.97726440429688


Epoch: 1: 100%|██████████| 69/69 [00:10<00:00,  6.67it/s]


Saving new best model with AUC: 0.7976624965667725
Loss: 0.1375710517168045, Accuracy: 95.19302368164062, AUC: 78.31536865234375, Validation Loss: 0.16289430856704712, Validation Accuracy: 94.15079498291016, Validation AUC: 79.76625061035156


Epoch: 2: 100%|██████████| 69/69 [00:10<00:00,  6.54it/s]


...

Loss: 0.06941673904657364, Accuracy: 97.48487091064453, AUC: 93.07058715820312, Validation Loss: 0.12111222743988037, Validation Accuracy: 95.58415985107422, Validation AUC: 88.28558349609375


Epoch: 48: 100%|██████████| 69/69 [00:10<00:00,  6.28it/s]


Loss: 0.06938845664262772, Accuracy: 97.46566772460938, AUC: 92.944580078125, Validation Loss: 0.12908776104450226, Validation Accuracy: 95.41915130615234, Validation AUC: 88.49449920654297


Epoch: 49: 100%|██████████| 69/69 [00:11<00:00,  6.24it/s]


Loss: 0.0672377273440361, Accuracy: 97.5546875, AUC: 93.30960845947266, Validation Loss: 0.11634033173322678, Validation Accuracy: 95.85112762451172, Validation AUC: 87.79619598388672


In [22]:
trainer = PTBXLTrainer(ensemble[3], model_name="full_supervised_inception_4", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

Epoch: 0: 100%|██████████| 69/69 [00:24<00:00,  2.83it/s]


Saving new best model with AUC: 0.7038715481758118
Loss: 0.25709274411201477, Accuracy: 90.87163543701172, AUC: 63.514347076416016, Validation Loss: 0.2409396469593048, Validation Accuracy: 90.76441192626953, Validation AUC: 70.38715362548828


Epoch: 1: 100%|██████████| 69/69 [00:11<00:00,  6.08it/s]


Saving new best model with AUC: 0.7981967926025391
Loss: 0.13688287138938904, Accuracy: 95.18801879882812, AUC: 78.22856140136719, Validation Loss: 0.15244872868061066, Validation Accuracy: 94.65715026855469, Validation AUC: 79.8196792602539


Epoch: 2: 100%|██████████| 69/69 [00:10<00:00,  6.35it/s]


...

Loss: 0.06572378426790237, Accuracy: 97.61642456054688, AUC: 93.39289855957031, Validation Loss: 0.12422023713588715, Validation Accuracy: 95.62996673583984, Validation AUC: 86.8600845336914


Epoch: 48: 100%|██████████| 69/69 [00:10<00:00,  6.62it/s]


Loss: 0.06703103333711624, Accuracy: 97.59178924560547, AUC: 92.98523712158203, Validation Loss: 0.1252318024635315, Validation Accuracy: 95.51223754882812, Validation AUC: 87.59748840332031


Epoch: 49: 100%|██████████| 69/69 [00:10<00:00,  6.63it/s]


Loss: 0.0669998824596405, Accuracy: 97.58993530273438, AUC: 93.41567993164062, Validation Loss: 0.1391758918762207, Validation Accuracy: 95.06253051757812, Validation AUC: 88.17906188964844


In [23]:
trainer = PTBXLTrainer(ensemble[4], model_name="full_supervised_inception_5", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

Epoch: 0: 100%|██████████| 69/69 [00:19<00:00,  3.56it/s]


Saving new best model with AUC: 0.7206538915634155
Loss: 0.26993635296821594, Accuracy: 90.29560089111328, AUC: 62.05104064941406, Validation Loss: 0.23138123750686646, Validation Accuracy: 91.38553619384766, Validation AUC: 72.06539154052734


Epoch: 1: 100%|██████████| 69/69 [00:10<00:00,  6.59it/s]


Saving new best model with AUC: 0.7993183732032776
Loss: 0.1380147933959961, Accuracy: 95.16690826416016, AUC: 78.28594207763672, Validation Loss: 0.15268833935260773, Validation Accuracy: 94.59902954101562, Validation AUC: 79.93183898925781


Epoch: 2: 100%|██████████| 69/69 [00:10<00:00,  6.54it/s]


...

Loss: 0.07229997217655182, Accuracy: 97.38713836669922, AUC: 92.23124694824219, Validation Loss: 0.12014611810445786, Validation Accuracy: 95.78462982177734, Validation AUC: 87.97293090820312


Epoch: 48: 100%|██████████| 69/69 [00:10<00:00,  6.64it/s]


Saving new best model with AUC: 0.8923133015632629
Loss: 0.07048401981592178, Accuracy: 97.43647003173828, AUC: 92.61746215820312, Validation Loss: 0.12172838300466537, Validation Accuracy: 95.56199645996094, Validation AUC: 89.23133087158203


Epoch: 49: 100%|██████████| 69/69 [00:10<00:00,  6.59it/s]


Loss: 0.07016405463218689, Accuracy: 97.45677947998047, AUC: 92.7608413696289, Validation Loss: 0.12268590927124023, Validation Accuracy: 95.69449615478516, Validation AUC: 87.4765396118164


## Testing

Test the Inception network.

In [24]:
import tensorflow as tf
from tensorflow import keras

In [27]:
loss_object = keras.losses.BinaryCrossentropy()
test_loss = keras.metrics.Mean(name="test_loss")
test_accuracy = keras.metrics.BinaryAccuracy(name="test_accuracy")
test_auc = keras.metrics.AUC(multi_label=True, name="test_auc", num_labels=num_classes)

@tf.function
def test_step_ensemble(time_series_batch, label_batch, models_to_test):
    ensemble_predictions = []
    for model_to_test in models_to_test:
        sliding_window_predictions = []
        for sliding_window in time_series_batch:
            sliding_window_predictions.append(model_to_test(sliding_window, training=False))
    
        ensemble_predictions.append(tf.reduce_max(sliding_window_predictions, axis=0))
    predictions = tf.reduce_mean(ensemble_predictions, axis=0)
    t_loss = loss_object(label_batch, predictions)

    test_loss(t_loss)
    test_accuracy(label_batch, predictions)
    test_auc(label_batch, predictions)

In [28]:
model_names = [f"full_supervised_inception_{x}" for x in range(1, 6)]

models = [keras.models.load_model(f"../models/{model_name}/best_model.keras") for model_name in model_names]

test_loss.reset_states()
test_accuracy.reset_states()
test_auc.reset_states()

for test_series, test_labels in data_module.test_dataset.batch(128):
    test_step_ensemble(test_series, test_labels, models)

print(
    f'Test data stats for model ensemble: '
    f'Loss: {test_loss.result()}, '
    f'Accuracy: {test_accuracy.result() * 100}, '
    f'AUC: {test_auc.result() * 100}'
)

Test data stats for model ensemble: Loss: 0.10352502763271332, Accuracy: 96.25025177001953, AUC: 90.32911682128906


## H-Inception network
Compare it to the H-Inception network, which has some handcrafted filters.

### Build the models

In [29]:
h_builder = InceptionNetworkBuilder(use_handcrafted_filters=True)
config = InceptionNetworkConfig(use_handcrafted_filters=True)
h_ensemble = [builder.build_model(config) for _ in range(5)]

### Train all H-Inception models

In [30]:
for i, model in enumerate(h_ensemble):
    trainer = PTBXLTrainer(model, model_name=f"full_supervised_h_inception_{i+1}", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
    trainer.fit(data_module.train_dataset, data_module.validation_dataset)

Epoch: 0: 100%|██████████| 69/69 [00:19<00:00,  3.52it/s]


Saving new best model with AUC: 0.7638120055198669
Loss: 0.2595270872116089, Accuracy: 90.56297302246094, AUC: 62.76792907714844, Validation Loss: 0.21065664291381836, Validation Accuracy: 93.89859771728516, Validation AUC: 76.3812026977539


Epoch: 1: 100%|██████████| 69/69 [00:10<00:00,  6.56it/s]


Saving new best model with AUC: 0.8027253746986389
Loss: 0.14013902842998505, Accuracy: 95.06128692626953, AUC: 79.09342956542969, Validation Loss: 0.1705813705921173, Validation Accuracy: 94.00696563720703, Validation AUC: 80.27253723144531


Epoch: 2: 100%|██████████| 69/69 [00:10<00:00,  6.64it/s]


...

Loss: 0.07102589309215546, Accuracy: 97.4014663696289, AUC: 92.65690612792969, Validation Loss: 0.10757479071617126, Validation Accuracy: 96.1565170288086, Validation AUC: 86.90760803222656


Epoch: 48: 100%|██████████| 69/69 [00:11<00:00,  6.14it/s]


Loss: 0.06830882281064987, Accuracy: 97.51332092285156, AUC: 92.73255920410156, Validation Loss: 0.11002341657876968, Validation Accuracy: 96.04322814941406, Validation AUC: 86.31951141357422


Epoch: 49: 100%|██████████| 69/69 [00:11<00:00,  6.16it/s]


Loss: 0.07068370282649994, Accuracy: 97.41023254394531, AUC: 92.2068099975586, Validation Loss: 0.11744621396064758, Validation Accuracy: 95.70779418945312, Validation AUC: 87.1795654296875


### Test models

In [31]:
model_names = [f"full_supervised_h_inception_{x}" for x in range(1, 6)]

models = [keras.models.load_model(f"../models/{model_name}/best_model.keras") for model_name in model_names]

test_loss.reset_states()
test_accuracy.reset_states()
test_auc.reset_states()

for test_series, test_labels in data_module.test_dataset.batch(128):
    test_step_ensemble(test_series, test_labels, models)

print(
    f'Test data stats for model ensemble: '
    f'Loss: {test_loss.result()}, '
    f'Accuracy: {test_accuracy.result() * 100}, '
    f'AUC: {test_auc.result() * 100}'
)

Test data stats for model ensemble: Loss: 0.10282047837972641, Accuracy: 96.36668395996094, AUC: 89.5983657836914
